# SOHO MDI and Sunspotter Scores

## Data Exploration 

### Dataset in a Nutshell

- Image data collected by SOHO/MDI
- MDI instrument data / SMART cutouts
- Citizen science project to label (some of) the data (around 60,000 images?): People asked to classify which of two images is more complex
- Complexity score computed from resulting rankings
- Dataset consists of 
    - metadata file (incl. score and image file name)
    - image files


### Download Data

Can be downloaded from XXX <br>


In [ ]:
!mkdir -p ./data/MDIComplexityScores
!curl -L 'https://dl.dropboxusercontent.com/s/le0isfa0r5c0w8z/sunspot_data.zip' > sunspot_data.zip
!unzip -q -o sunspot_data.zip -d ./data/MDIComplexityScores
!rm -f sunspot_data.zip

Install dependencies

- on Google Colab you will need to restart your runtime after this step
- when running this locally, make sure to first follow the setup instructions in the [README](../README.md)

In [ ]:
#Install dependencies

#- on Google Colab you will need to restart your runtime after this step
#- when running this locally, make sure to first follow the setup instructions in the [README](../README.md)!pip install -U tqdm
!pip install -U torch torchvision
!pip install -U numpy
!pip install -U matplotlib 
!pip install -U Pillow
!pip install -U pandas

In [ ]:
# Data location
DATADIR = "./data/MDIComplexityScores/data/images/"
METADATA_FILE = "./data/MDIComplexityScores/data/image_metadata.csv"

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

### Load Data Metadata

In [ ]:
metadata = pd.read_csv(METADATA_FILE)
metadata.tail()

Here we are only interested in the `score` variable.

In [ ]:
score = metadata.score
score

### First Glance at a Sample Image

In [ ]:
samplefile = metadata.filename_x[0]
image = Image.open(DATADIR + samplefile)
imgarray = np.asarray(image)

print(image.size, image.mode)
print(imgarray.shape)
print(imgarray.min(), imgarray.max())
image

### Load and preprocess the images

- Load image, possibly resize with a suitable ratio.
- Average over color channels.
- Rescale intensity scale to be between `[0,1]`
- Rearrange to have the channels in the first dimension

Data are stored in the format (image_no, x_pixel, y_pixel)

In [ ]:
def preprocess_data( image_path, ratio=1 ):

    # create empty data cube
    data = []

    for file in tqdm( metadata.filename_x ):

        # Load and resize image
        image = Image.open(image_path + file)
        new_size = (np.array( image.size ) / ratio).astype( int )
        image = image.resize( new_size )

        # compute the mean along the color channel
        image = np.mean( image, axis=2 )

        # append the image to data list
        data.append( image )
    
    # stack elements of the list into a data cube
    data = np.dstack( data )
    
    # permute dimensions to better format 
    data = np.transpose( data, axes=(2,0,1) )
    
    # make sure pixel values are between 0 and 1
    data = data / 255
    
    return data

In [ ]:
data = preprocess_data( DATADIR, ratio=4 )
print( "Data shape: {}".format( data.shape ) )

### Plot Many Examples

In [ ]:
plt.figure( figsize=(20,20) )
n = 10
m = 10
for i in range(n):
    for j in range(m):
        plt.subplot( n, m, n*i+j+1 )
        image = np.random.randint( data.shape[0] )
        plt.title( "score: {}".format( np.round( metadata.score[image]) ) )
        plt.imshow( data[image,:,:], cmap="gray" )
        plt.axis('off')

### Complexity score distribution

In [ ]:
plt.hist( score )

### Split into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, score, test_size=0.20, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Modeling the Data

In [ ]:
import torch

X_train_dense = X_train.reshape( X_train.shape[0], -1 )    
X_test_dense = X_test.reshape( X_test.shape[0], -1 )
X_train_dense.shape, X_test_dense.shape

batch_size = 32

X_train_dense_split = torch.split(torch.tensor(X_train_dense).float(),batch_size)
y_train_split = torch.split(torch.tensor(y_train.values).float(),batch_size)

X_test_dense_split = torch.split(torch.tensor(X_test_dense).float(),batch_size)
y_test_split = torch.split(torch.tensor(y_test.values).float(),batch_size)


### Training

In [ ]:
from torch.nn import Sequential, Linear, ReLU
from torch.optim import Adam
from torch.nn import MSELoss


model = Sequential(
  Linear(X_train_dense.shape[1], 500), ReLU(),
  Linear(500, 300), ReLU(),
  Linear(300, 1) )

optim = Adam(model.parameters())
mse = MSELoss()

losses_train = []
losses_test = []

for i in range (5): 
   ltr = 0.0
   lte = 0.0
   print ("Epoch: "+str(i))
   for j in range(len(X_train_dense_split)):
      model.zero_grad()
      result = model(X_train_dense_split[j]).view(-1) 
      loss = mse(result,y_train_split[j])
      loss.backward()
      optim.step()
      ltr+=loss.detach()
   for j in range(len(X_test_dense_split)):
      result = model(X_test_dense_split[j]).view(-1)
      loss = mse(result,y_test_split[j])
      lte+=loss.detach()
   losses_train.append(ltr/len(X_train_dense_split))
   losses_test.append(lte/len(X_test_dense_split))
    
print ("done")

### Learning curve

In [ ]:
plt.plot( losses_train, label='training' )
plt.plot( losses_test, label='test' )
axes = plt.gca()
axes.set_ylim([5000,20000])
plt.legend()

### Predictions

In [ ]:
y_test_pred = model( torch.tensor(X_test_dense).float() ).detach().numpy() 

### Plotting predictions

In [ ]:
plt.scatter( y_test_pred, y_test )
plt.xlabel("predicted")
plt.ylabel("true")

### Scores

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
rmse = np.sqrt( mean_squared_error( y_test, y_test_pred ) )
r2 = r2_score( y_test, y_test_pred )
print("RMSE = {}, R2 = {}".format( rmse, r2 ) )

## CNN Model

In [ ]:
import numpy as np

from torch.nn import Sequential, Conv2d, Linear, Flatten, ReLU
from torch.optim import Adam
from torch.nn import MSELoss


model = Sequential(
  Conv2d(1, 16, 3, 1, padding=1), ReLU(),
  Conv2d(16, 16, 3, 2, padding=1), ReLU(),
  Conv2d(16, 32, 3, 1, padding=1), ReLU(),
  Conv2d(32, 32, 3, 2, padding=1), ReLU(),
  Flatten(),
  Linear(13248, 1000), 
  Linear(1000, 800), 
  Linear(800, 500), 
  Linear(500, 100), 
  Linear(100,1)
)

optim = Adam(model.parameters())
mse = MSELoss()


In [ ]:
X_train_cnn = torch.tensor( np.expand_dims( X_train, axis=1 )).float()
X_test_cnn =  torch.tensor( np.expand_dims( X_test, axis=1 )).float()

batch_size = 32

X_train_cnn_split = torch.split(X_train_cnn,batch_size)
X_test_cnn_split = torch.split(X_test_cnn,batch_size)


print (X_train_cnn.shape)

### Training

In [ ]:

losses_train = []
losses_test = []


for i in range (5): 
   print ("Epoch: "+str(i))
   ltr = 0.0
   lte = 0.0
   for j in range(len(X_train_cnn_split)):
      model.zero_grad()
      result = model(X_train_cnn_split[j]).view(-1) 
      loss = mse(result,y_train_split[j])
      loss.backward()
      optim.step()
      ltr+=loss.detach()
   for j in range(len(X_test_cnn_split)):
      result = model(X_test_cnn_split[j]).view(-1)
      loss = mse(result,y_test_split[j])
      lte+=loss.detach()
   losses_train.append(ltr/len(X_train_dense_split))
   losses_test.append(lte/len(X_test_dense_split))
    
print ("done")

### Learning curve

In [ ]:
plt.plot( losses_train, label='training' )
plt.plot( losses_test, label='test' )
plt.legend()

### Predictions

In [ ]:
y_test_pred = model( X_test_cnn.float() ).detach().numpy() 

### Plotting predictions

In [ ]:
plt.scatter( y_test_pred, y_test )
plt.xlabel("predicted")
plt.ylabel("true")

### Scores

In [ ]:

from sklearn.metrics import r2_score, mean_squared_error
rmse = np.sqrt( mean_squared_error( y_test, y_test_pred ) )
r2 = r2_score( y_test, y_test_pred )
print("RMSE = {}, R2 = {}".format( rmse, r2 ) )